# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2023
topic: str = "MEDICINA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [4]:
add_list: list = [27, 28] + [x for x in range(33, 40, 1)] + [43, 46, 75, 129, 146, 151, 162, 170]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,1. Pregunta asociada a la imagen 1.
1,Hombre d e 56 a ños con un t umor p alpable e...
2,lado izquierdo del cuello móvil a la deglución...
3,acompaña d e di sfagia y d isfonía. Tiene hist...
4,familiar de tumo res endocrinos. ¿Cuál de l os
5,siguientes es el diagnóstico anatomopatológico...
6,probable de la lesión que aparece en la imagen?:
7,1. Carcinoma papilar de tiroides.
8,2. Carcinoma medular de tiroides.
9,3. Linfoma difuso de célula grande.


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [6]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,2,43,1.0,85,1.0,127,1,169,3.0
1,2,4,44,4.0,86,1.0,128,2,170,1.0
2,3,2,45,4.0,87,2.0,129,4,171,4.0
3,4,3,46,2.0,88,4.0,130,3,172,2.0
4,5,3,47,3.0,89,2.0,131,3,173,1.0
5,6,2,48,3.0,90,1.0,132,4,174,2.0
6,7,4,49,2.0,91,4.0,133,1,175,1.0
7,8,3,50,3.0,92,1.0,134,3,176,3.0
8,9,1,51,1.0,93,3.0,135,1,177,3.0
9,10,4,52,4.0,94,2.0,136,4,178,2.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [7]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [8]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [9]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [10]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [11]:
raw_exam_df.head(10)

0                  1. Pregunta asociada a la imagen 1.
1    Hombre d e 56  a ños con un t umor p alpable e...
2    lado izquierdo del cuello móvil a la deglución...
3    acompaña d e di sfagia y d isfonía. Tiene hist...
4       familiar de tumo res endocrinos. ¿Cuál de l os
5    siguientes es el diagnóstico anatomopatológico...
6     probable de la lesión que aparece en la imagen?:
7                    1. Carcinoma papilar de tiroides.
8                    2. Carcinoma medular de tiroides.
9                  3. Linfoma difuso de célula grande.
Name: text, dtype: object

In [12]:
raw_exam_df.tail(10)

3055      2. No se puede cumplimentar el CMD por tratarse
3056                de muerte sospechosa de criminalidad.
3057      3. No se puede cumplimentar el CMD por que sólo
3058     se di spone de  cri terios de presunción y no de
3059        confirmación, au nque la s circunstancias del
3060                fallecimiento y la patobiografía sean
3061                                         compatibles.
3062    4. Se hará constar la infección por COVID-19 e...
3063           apartado de causa fund amental y no e n el
3064                         apartado de causa inmediata.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [13]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [14]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. Pregunta asociada a la imagen 1. Hombre d e...
1                    1. Carcinoma papilar de tiroides.
2                    2. Carcinoma medular de tiroides.
3                  3. Linfoma difuso de célula grande.
4                                 4. Sarcoma sinovial.
5    2. Pregunta asociada a la imagen 2. Paciente q...
6                 1. Fractura panfacial tipo Lefort I.
7                2. Fractura panfacial tipo Lefort II.
8         3. Fractura mandibular bicondílea inestable.
9    4. Fractura ma ndibular bilateral en ramas hor...
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [15]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 27
Number of rows not ending with '.', ':' or '?' = 33
[55, 81, 242, 269, 270, 271, 272, 288, 289, 295, 336, 364, 366, 368, 375, 386, 498, 506, 509, 623, 634, 665, 752, 837, 859, 894, 905, 916, 917, 933, 944, 967, 1041]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [16]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [17]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [18]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [55, 81, 242, 288, 289, 295, 336, 364, 366, 368, 375, 386, 506, 623, 634, 665, 752, 837, 859, 894, 905, 916, 917, 933, 944, 967, 1041]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 55:
53                         3. Pseudoaneurisma de aorta.
54            4. Disección de aorta tipo A de Stanford.
55    12. Pregunta asociada a la imagen 12. Hombre d...
56    0,95 cm2 (imagen). ¿Cuál es el diagnóstico más...
57    1. Estenosis valvular aórtica grave con indica...
Name: text, dtype: object
Id to fix 81:
79                                    3. Esquistocitos.
80                                      4. Acantocitos.
81    17. Pregunta asociada a la imagen 17. Hombre d...
82    180/110 mm Hg, iniciá ndose tratamiento con en...
83    1. Tiene una estenosis unilateral de la arteri...
Name: text, dtype: object
Id to fix 242:
240                                      3. Acamprosato.
241                                          4. Tiamina.
242    49. Un artículo describe un ensayo clínico ale...
243    42 centr os sanitari os de  16 países dis tint...
244    1. Al ser una investigación alea torizada, el ...
Name: text, dtype: object
Id to fix 269:
267                

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [19]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 1050
1050


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [20]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [21]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [22]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [23]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [24]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. Pregunta asociada a la imagen 1. Hombre d e...,1. Carcinoma papilar de tiroides.,2. Carcinoma medular de tiroides.,3. Linfoma difuso de célula grande.,4. Sarcoma sinovial.
1,2. Pregunta asociada a la imagen 2. Paciente q...,1. Fractura panfacial tipo Lefort I.,2. Fractura panfacial tipo Lefort II.,3. Fractura mandibular bicondílea inestable.,4. Fractura ma ndibular bilateral en ramas hor...
2,3. Pregunta asociada a la imagen 3. Hombre de ...,1. Dermatitis seborreica.,2. Queratosis actínica.,3. Liquen nitidus.,4. Psoriasis.
3,4. Pregunta asociada a la imagen 4. Hombre de ...,1. Condiloma lata.,2. Molluscum contagiosum.,3. Condiloma acuminado.,4. Liquen plano.
4,5. Pregunta asociada a la imagen 5. Mujer de 3...,1. El tratamiento electivo de inicio es tumore...,2. El tratamiento electivo de inicio es mastec...,3. Se vis ualiza afectación en mama izquie rda...,"4. Se visualiza afectación en mama izquierda, ..."
5,6. Pregunta asociada a la imagen 6. Mujer de 2...,1. La pérdida de peso no es eficaz en esta sit...,2. La acetazolamida y la furosemida s on fárma...,3. Se debe realizar una punción lumbar a ntes ...,4. Los corticoides orales se usan ampliamente ...
6,7. Pregunta asociada a la imagen 7. Mujer de 5...,1. RM sin contraste. Meningioma.,2. TC con contraste. Tumor epidermoide.,3. RM con contraste. Tumor glial.,4. RM con contraste. Schwannoma vestibular.
7,8. Pregunta asociada a la imagen 8. Hombre de ...,1. Absceso cerebral secundario a otitis media ...,2. Trombosis venosa del seno sigmoide derecho.,3. Infarto i squémico ag udo e n t erritorio d...,4. Neoplasia ce rebral de origen glial c on e...
8,9. Pregunta asociada a la imagen 9. Hombre de ...,1. Obstrucción intestinal secunda ria a neopla...,2. Obstrucción intestinal d e cau sa ex trínse...,3. Obstrucción intestinal por mi gración e imp...,4. Obstrucción funcional o íleo pseudoobstruct...
9,10. Pregunta asociada a la imagen 10. Hombre d...,1. Neumonía por Mycobacterium tuberculosis.,2. Neumonía por Aspergillus.,3. Neumonía por Streptococcus pneumoniae.,4. Neumonía por Pneumocystis jirovecii.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [25]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [26]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [27]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [28]:
answers_df_clean.head(10)

,V,RC
0,1,2.0
1,2,4.0
2,3,2.0
3,4,3.0
4,5,3.0
5,6,2.0
6,7,4.0
7,8,3.0
8,9,1.0
9,10,4.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [29]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. Pregunta asociada a la imagen 1. Hombre d e...,1. Carcinoma papilar de tiroides.,2. Carcinoma medular de tiroides.,3. Linfoma difuso de célula grande.,4. Sarcoma sinovial.,2.0
1,2. Pregunta asociada a la imagen 2. Paciente q...,1. Fractura panfacial tipo Lefort I.,2. Fractura panfacial tipo Lefort II.,3. Fractura mandibular bicondílea inestable.,4. Fractura ma ndibular bilateral en ramas hor...,4.0
2,3. Pregunta asociada a la imagen 3. Hombre de ...,1. Dermatitis seborreica.,2. Queratosis actínica.,3. Liquen nitidus.,4. Psoriasis.,2.0
3,4. Pregunta asociada a la imagen 4. Hombre de ...,1. Condiloma lata.,2. Molluscum contagiosum.,3. Condiloma acuminado.,4. Liquen plano.,3.0
4,5. Pregunta asociada a la imagen 5. Mujer de 3...,1. El tratamiento electivo de inicio es tumore...,2. El tratamiento electivo de inicio es mastec...,3. Se vis ualiza afectación en mama izquie rda...,"4. Se visualiza afectación en mama izquierda, ...",3.0
5,6. Pregunta asociada a la imagen 6. Mujer de 2...,1. La pérdida de peso no es eficaz en esta sit...,2. La acetazolamida y la furosemida s on fárma...,3. Se debe realizar una punción lumbar a ntes ...,4. Los corticoides orales se usan ampliamente ...,2.0
6,7. Pregunta asociada a la imagen 7. Mujer de 5...,1. RM sin contraste. Meningioma.,2. TC con contraste. Tumor epidermoide.,3. RM con contraste. Tumor glial.,4. RM con contraste. Schwannoma vestibular.,4.0
7,8. Pregunta asociada a la imagen 8. Hombre de ...,1. Absceso cerebral secundario a otitis media ...,2. Trombosis venosa del seno sigmoide derecho.,3. Infarto i squémico ag udo e n t erritorio d...,4. Neoplasia ce rebral de origen glial c on e...,3.0
8,9. Pregunta asociada a la imagen 9. Hombre de ...,1. Obstrucción intestinal secunda ria a neopla...,2. Obstrucción intestinal d e cau sa ex trínse...,3. Obstrucción intestinal por mi gración e imp...,4. Obstrucción funcional o íleo pseudoobstruct...,1.0
9,10. Pregunta asociada a la imagen 10. Hombre d...,1. Neumonía por Mycobacterium tuberculosis.,2. Neumonía por Aspergillus.,3. Neumonía por Streptococcus pneumoniae.,4. Neumonía por Pneumocystis jirovecii.,4.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [30]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [31]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [32]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [33]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. Pregunta asociada a la imagen 1. Hombre d e...,1. Carcinoma papilar de tiroides.,2. Carcinoma medular de tiroides.,3. Linfoma difuso de célula grande.,4. Sarcoma sinovial.,2,2023
1,2. Pregunta asociada a la imagen 2. Paciente q...,1. Fractura panfacial tipo Lefort I.,2. Fractura panfacial tipo Lefort II.,3. Fractura mandibular bicondílea inestable.,4. Fractura ma ndibular bilateral en ramas hor...,4,2023
2,3. Pregunta asociada a la imagen 3. Hombre de ...,1. Dermatitis seborreica.,2. Queratosis actínica.,3. Liquen nitidus.,4. Psoriasis.,2,2023
3,4. Pregunta asociada a la imagen 4. Hombre de ...,1. Condiloma lata.,2. Molluscum contagiosum.,3. Condiloma acuminado.,4. Liquen plano.,3,2023
4,5. Pregunta asociada a la imagen 5. Mujer de 3...,1. El tratamiento electivo de inicio es tumore...,2. El tratamiento electivo de inicio es mastec...,3. Se vis ualiza afectación en mama izquie rda...,"4. Se visualiza afectación en mama izquierda, ...",3,2023
5,6. Pregunta asociada a la imagen 6. Mujer de 2...,1. La pérdida de peso no es eficaz en esta sit...,2. La acetazolamida y la furosemida s on fárma...,3. Se debe realizar una punción lumbar a ntes ...,4. Los corticoides orales se usan ampliamente ...,2,2023
6,7. Pregunta asociada a la imagen 7. Mujer de 5...,1. RM sin contraste. Meningioma.,2. TC con contraste. Tumor epidermoide.,3. RM con contraste. Tumor glial.,4. RM con contraste. Schwannoma vestibular.,4,2023
7,8. Pregunta asociada a la imagen 8. Hombre de ...,1. Absceso cerebral secundario a otitis media ...,2. Trombosis venosa del seno sigmoide derecho.,3. Infarto i squémico ag udo e n t erritorio d...,4. Neoplasia ce rebral de origen glial c on e...,3,2023
8,9. Pregunta asociada a la imagen 9. Hombre de ...,1. Obstrucción intestinal secunda ria a neopla...,2. Obstrucción intestinal d e cau sa ex trínse...,3. Obstrucción intestinal por mi gración e imp...,4. Obstrucción funcional o íleo pseudoobstruct...,1,2023
9,10. Pregunta asociada a la imagen 10. Hombre d...,1. Neumonía por Mycobacterium tuberculosis.,2. Neumonía por Aspergillus.,3. Neumonía por Streptococcus pneumoniae.,4. Neumonía por Pneumocystis jirovecii.,4,2023


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [34]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 17
Number of rows in filtered df = 17


**2. Showing the filtered DataFrame**

Using head and tail

In [35]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,27. ¿Cuál de las siguientes alteraciones bioqu...,1. La disminuci ón de la traslocación de los ...,2. El aumento de la producción de urea por una...,3. El incre mento de la síntesis he pática de ...,4. El aume nto de la síntesis hepática de glic...,3,2023
1,28. Una persona sana s ufre una des hidratació...,1. Los cambios de presión arterial se acompaña...,2. Se produce vasoconstricción de las arte rio...,3. Aumenta la activ idad d e los núcleos hipot...,4. Aumenta la secreción renal de potasio.,1,2023
2,33. Una de las siguientes enfermedades se prod...,1. Enfermedad de Hungtinton.,2. Ataxia de Friedreich.,3. Síndrome del cromosoma X frágil.,4. Distrofia miotónica tipo 1.,3,2023
3,34. Un rasgo que aparece e n todas las gene...,1. Autosómica dominante.,2. Ligada al sexo dominante.,3. Ligada al sexo recesiva.,4. Mitocondrial.,2,2023
4,35. Las células tumorales desarrollan diferent...,1. La i noculación intratumoral de cél ulas T ...,2. La inoculación de células de la inmunidad i...,3. El uso de citocinas como el TGF-beta o la I...,4. El uso de anticuerpos monoclonales inmunomo...,4,2023
5,36. En relación con el m ecanismo inmunol ógic...,1. El rechazo agudo mediado por linfocitos T p...,2. La prese ncia en el receptor de a nticuerpo...,3. El rechazo crónico o lesión crónica del inj...,4. El rechazo hiperagudo es mediado por linfoc...,4,2023
6,37. La vacunación utilizando ARN mensa jero de...,1. La r espuesta n o requiere l a c olaboració...,2. Un ele vado porce ntaje de pacientes inmuno...,3. Es una forma de inmuniz ación activa cuyo r...,4. Induce memoria inmuno lógica b asada en la ...,1,2023
7,38. En relación con las enfer medades as ociad...,1. La presencia de anticue rpos frente al rece...,2. La prese ncia de autoanticuerpos antidesmog...,3. La miastenia gravis se asocia a la presenc...,4. El síndrome de G oodpasture se asoci a a l ...,1,2023
8,39. En el asma bronquial son numer osas las cé...,1. Los lin focitos B so n los responsables de ...,2. En el asma bronquial hay un predominio de f...,3. Las citocinas IL4 e IL 13 son las fa vorece...,4. La población de linfocitos T reguladores (C...,3,2023
9,43. En relación con las condiciones necesarias...,1. Debe e xistir una prueba adecu ada para l a...,2. La enfermeda d que se detecta mediant e el ...,3. La enfermeda d que se detecta mediant e el ...,4. Se debe disponer de recursos suficientes pa...,1,2023


In [36]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
7,38. En relación con las enfer medades as ociad...,1. La presencia de anticue rpos frente al rece...,2. La prese ncia de autoanticuerpos antidesmog...,3. La miastenia gravis se asocia a la presenc...,4. El síndrome de G oodpasture se asoci a a l ...,1,2023
8,39. En el asma bronquial son numer osas las cé...,1. Los lin focitos B so n los responsables de ...,2. En el asma bronquial hay un predominio de f...,3. Las citocinas IL4 e IL 13 son las fa vorece...,4. La población de linfocitos T reguladores (C...,3,2023
9,43. En relación con las condiciones necesarias...,1. Debe e xistir una prueba adecu ada para l a...,2. La enfermeda d que se detecta mediant e el ...,3. La enfermeda d que se detecta mediant e el ...,4. Se debe disponer de recursos suficientes pa...,1,2023
10,46. En un estudio s obre la validez de un a p...,1. El índice de Kappa.,2. La curv a ROC (receiver operating character...,3. El coeficiente de correlación intraclase.,4. El gráfico de Altman y Bland.,2,2023
11,75. Acerca de la bronquiolitis aguda en la inf...,1. El v irus resp iratorio sincitial (VRS) es ...,2. Se recomienda la administración de palivizu...,3. La bro nquiolitis se produce po r obstrucci...,4. El tratamient o se basa e n aeros oles de...,2,2023
12,129. ¿Cuál de lo s siguientes genes puede es t...,1. RAS.,2. p53.,3. BRCA1.,4. MLH1.,4,2023
13,"146. En relación c on el cáncer de prós tata, ...",1. El principal factor de riesgo es la edad.,2. Los por tadores del gen BR CA-1 tienen mayo...,3. Es más frecuente en varones caucásicos.,"4. Las grasas animales, la ob esidad, el háb i...",3,2023
14,151. ¿Cuál de las si guientes al teraciones e...,1. Mutación de KRAS.,2. Mutación de BRAF.,3. Reordenamiento de ALK.,4. Fusión de NTRK.,1,2023
15,162. Respecto a los c ambios d el sistema in m...,1. La di sminución de i nterleuquina 6 se rela...,2. Disminuye el número de linfocitos T.,3. Disminuye el factor de necrosis tumoral.,4. Disminuye la producción de autoanticuerpos ...,2,2023
16,170. El tratamiento de la leishmaniasis viscer...,1. Anfotericina B liposomal.,2. Voriconazol y anfotericina B liposomal.,3. Anfotericina B liposomal y miltefosina.,4. Miltefosina y paromomicina.,1,2023


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [37]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [38]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [39]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""


**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [40]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()